In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
df=pd.read_csv("sentiment.csv")
df.head(10)

,Index,message to examine,label (depression result)
0,106,just had a real good moment. i missssssssss hi...,0
1,217,is reading manga http://plurk.com/p/mzp1e,0
2,220,@comeagainjen http://twitpic.com/2y2lx - http:...,0
3,288,@lapcat Need to send 'em to my accountant tomo...,0
4,540,ADD ME ON MYSPACE!!! myspace.com/LookThunder,0
5,624,so sleepy. good times tonight though,0
6,701,"@SilkCharm re: #nbn as someone already said, d...",0
7,808,23 or 24ï¿½C possible today. Nice,0
8,1193,nite twitterville workout in the am -ciao,0
9,1324,"@daNanner Night, darlin'! Sweet dreams to you",0


In [3]:
# Missing values handling
df.isnull().sum()

Index                        0
message to examine           0
label (depression result)    0
dtype: int64

In [4]:
# Encoding using Tfidf vectorizer
tfidf_vectorizer=TfidfVectorizer(max_features=5000)
x=tfidf_vectorizer.fit_transform(df["message to examine"])
y=df["label (depression result)"]

In [5]:
# Conver the sparse tfidf matrix to a dense Numpy array
x=x.toarray()

In [6]:
x_train, x_test, y_train, y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [7]:
# Build the ANN model
model=keras.Sequential([keras.layers.Dense(32,activation="relu",input_shape=(x_train.shape[1],)),
                       keras.layers.Dense(64,activation="relu"),
                       keras.layers.Dense(128,activation="relu"),
                       keras.layers.Dense(256,activation="relu"),
                       keras.layers.Dense(512,activation="relu"),
                       keras.layers.Dense(1,activation="sigmoid")])

In [8]:
# Compile the emodel
model.compile(optimizer="adam", loss="binary_crossentropy",metrics=["accuracy"])

In [9]:
#$ Train the model
history=model.fit(x_train,y_train,epochs=5,batch_size=30,validation_data=(x_test,y_test))

Epoch 1/5
276/276 [==============================] - 2s 5ms/step - loss: 0.1963 - accuracy: 0.9160 - val_loss: 0.0620 - val_accuracy: 0.9825
Epoch 2/5
276/276 [==============================] - 1s 4ms/step - loss: 0.0132 - accuracy: 0.9964 - val_loss: 0.0806 - val_accuracy: 0.9753
Epoch 3/5
276/276 [==============================] - 1s 4ms/step - loss: 0.0021 - accuracy: 0.9998 - val_loss: 0.0995 - val_accuracy: 0.9796
Epoch 4/5
276/276 [==============================] - 1s 4ms/step - loss: 0.0023 - accuracy: 0.9998 - val_loss: 0.0781 - val_accuracy: 0.9806
Epoch 5/5
276/276 [==============================] - 1s 4ms/step - loss: 0.0020 - accuracy: 0.9998 - val_loss: 0.1262 - val_accuracy: 0.9753


In [10]:
# Evaluate the model on the test data
test_loss, test_acc=model.evaluate(x_test,y_test)
print(f"Test Loss : {test_loss : .4f}")
print(f"Test Accuracy : {test_acc : .4f}")

65/65 [==============================] - 0s 2ms/step - loss: 0.1262 - accuracy: 0.9753
Test Loss :  0.1262
Test Accuracy :  0.9753


In [11]:
# Save the model
model.save("senti.keras")

In [12]:
# Load the saved model
loaded_model=keras.models.load_model("senti.keras")
loaded_model